# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install

In [2]:
!pip install -U datasets==2.20.0 bitsandbytes accelerate transformers peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 18.0 MB/s eta 0:00:00
  Using cached 

# 7.4 QLoRA 미세조정

## 모델

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
)
import torch

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1"
)

gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## 데이터셋

In [11]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")
dataset['train'][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [12]:
doc = dataset["train"]['question'][0]
messages = [{"role": "user", "content": doc}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

<bos><start_of_turn>user
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
북태평양 기단과 오호츠크 해 기단은 북극에서 서쪽으로 멀리 떨어져 있기 때문에 미국 내에서 지속적으로 존재하지 않습니다. 북태평양 기단은 북극을 가로질러 멀리 동쪽으로 떠나지 않아 1년에 한 번만 미국 내에서 지속적으로 존재합니다. 오호츠크 해 기단도 동쪽으로 멀리 떨어져 있기 때문에 일반적으로 1년에 한 번만 미국 내에서 지속적으로 존재합니다. 하지만 이 두 기단은 시간을 끌어내어 결국 지나갑니다.<eos>


## QLoRA 미세조정 준비

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=bnb_config,
)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
def preprocess(example):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": example['question']},
            {"role": "model", "content": example['answers']['text'][0]},
        ],
        return_dict=True,
    )

dataset = dataset.map(
    preprocess,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)

## QLoRA 학습

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Books/outputs/ckpt",
    max_steps=3000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_ratio=0.04,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=collator,
)
trainer.train()

## 모델 저장

In [17]:
model.save_pretrained("/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1")

In [18]:
!ls -alh /content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1

total 38M
-rw------- 1 root root  726 Jul 10 16:51 adapter_config.json
-rw------- 1 root root  38M Jul 10 16:51 adapter_model.safetensors
-rw------- 1 root root 5.0K Jul 10 16:51 README.md


## 추론

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

base_model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Books/outputs/peft-a")
# 다른 어댑터 로드
model.load_adapter("/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1", adapter_name="a")
# 활성 어댑터 가중치 설정
model.set_adapter("a")

tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
)

# 이전에 만들었던 inputs, gen_cfg 사용
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))